<a href="https://colab.research.google.com/github/wenjie-hoo/baltic_ml/blob/main/baltic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from zipfile import ZipFile
from functools import reduce

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


In [ ]:
!pip install -q gdown httpimport
!gdown 'https://github.com/wenjie-hoo/baltic_ml/raw/main/2022.11.07-merged-single-observation.zip'

In [ ]:
zip_file = ZipFile('./2022.11.07-merged-single-observation.zip')
dfs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()
       if text_file.filename.endswith('.csv') and not text_file.filename[37:].startswith('.')}

df_list=[]
for df in dfs.values():
       df['DATE'] = pd.to_datetime(df['DATE'],dayfirst = True)
       df_list.append(df)

df = reduce(lambda left,right: pd.merge(left.drop_duplicates(subset=['DATE','DEPTH']),right.drop_duplicates(subset=['DATE','DEPTH']),on=['DATE','DEPTH'],how='outer'), df_list)
# df.to_csv('xxxxx.csv')
df.head

In [ ]:
# df.describe()
df.groupby('DATE').mean()['TP'].plot()

In [ ]:
## helpful filters and statistical functions

def fix_date_formatting(df):
    df['DATE'] = pd.to_datetime(df.DATE, infer_datetime_format=True)
    return df

def filter_by_months(df, months):
    return df[df['DATE'].dt.month == any(months)]

def group_by_year(df, par):
    df =  df.groupby(df['DATE'].dt.year)[par].mean()
    return pd.DataFrame({"DATE":df.index, par:df.values})

def group_by_date(df, par):
    df =  df.groupby(df['DATE'].dt.date)[par].mean()
    return pd.DataFrame({"DATE":df.index, par:df.values})

def group_by_month(df, par):
    df = df.drop('DEPTH', axis=1)
    df['DATE'] = pd.to_datetime(df['DATE'])
    df = df.groupby(pd.Grouper(freq='M', key='DATE')).mean()
    df = df.reset_index()
    return df

def filter_by_depth(df, depth):
    df = df.loc[df['DEPTH'] == depth]
    return df

def filter_by_depth_range(df, low, high):
    df = df.loc[low <= df['DEPTH'] <= high]
    return df

def group_by_depth(df, par):
    df =  df.groupby(df['DEPTH'])[par].mean()
    return pd.DataFrame({"DEPTH":df.index, par:df.values})

def drop_outliers(df,param, quantile):
    q = df[param].quantile(quantile)
    return df[df[param] < q]

#return new dataframe with par replaced by it's movign average
def moving_averages(df, param, window_size):
    _df = df.copy()
    _df[param] = df[param].rolling(window=window_size).mean()
    return _df

def apply_features_transform(df, param, quantile=.95, depth=[2000, 2500], moving_avg_window=6):
    df = fix_date_formatting(df)
    df = filter_by_depth_range(df, depth[0], depth[1])
    #df = df.drop('DEPTH', axis=1)
    df = drop_outliers(df, param, quantile)
    df = group_by_month(df, param)
    df = moving_averages(df, param, moving_avg_window)
    #df = group_by_year(df, param)
    
    return df

In [ ]:
BASE = './2022.11.07-merged-single-observation/'

# Chlorophyll a 
# CHLORA
chlora = pd.read_csv(BASE + 'CHLORA.csv', infer_datetime_format=True)
chlora = fix_date_formatting(chlora)

# Salinity: The amount of salt per amount of a body of water.
# CTDSAL
ctdsal = pd.read_csv(BASE + 'CTDSAL.csv', infer_datetime_format=True)
ctdsal = fix_date_formatting(ctdsal)

# Water temperature at measurement depth
# CTDTMP
ctdtmp = pd.read_csv(BASE + 'CTDTMP.csv', infer_datetime_format=True)
ctdtmp = fix_date_formatting(ctdtmp)

# Ammonia
# NH4
NH4 = pd.read_csv(BASE + 'NH4.csv', infer_datetime_format=True)
NH4 = fix_date_formatting(NH4)

# Nitrite 
# NO2
NO2 = pd.read_csv(BASE + 'NO2.csv', infer_datetime_format=True)
NO2 = fix_date_formatting(NO2)

# Nitrate 
# NO3
NO3 = pd.read_csv(BASE + 'NO3.csv', infer_datetime_format=True)
NO3 = fix_date_formatting(NO3)

# Dissolved oxygen  
# OXY
oxygen = pd.read_csv(BASE + 'OXY.csv', infer_datetime_format=True)
oxygen = fix_date_formatting(oxygen)

# PH
PH = pd.read_csv(BASE + 'PH.csv', infer_datetime_format=True)
PH = fix_date_formatting(PH)

# Phosphate 
# PO4
PO4 = pd.read_csv(BASE + 'PO4.csv', infer_datetime_format=True)
PO4 = fix_date_formatting(PO4)

# Used to measure water transparency or turbidity in bodies of water.
# The disc is mounted on a pole or line,
# and lowered slowly down in the water
# SECCHI
secchi = pd.read_csv(BASE + 'SECCHI.csv', infer_datetime_format=True)
secchi = fix_date_formatting(secchi)

# Silicate
# SIO2
SIO2 = pd.read_csv(BASE + 'SIO2.csv', infer_datetime_format=True)
SIO2 = fix_date_formatting(SIO2)

# Total nitrogen 
# TN
TN = pd.read_csv(BASE + 'TN.csv', infer_datetime_format=True)
TN = fix_date_formatting(TN)

# Total phosphorus 
# TP
TP = pd.read_csv(BASE + 'TP.csv', infer_datetime_format=True)
TP = fix_date_formatting(TP)

In [ ]:
depths = [ 100, 500, 1000, 1500, 2000, 2500]

## Some theory water exchange

Baltic sea is  not very salty naturally and it's saltiness levels are dependent on water exchange with global ocean by north sea. which in the case of the Baltic Sea is difficult due to narrow and tight connections (Danish straits)

water exchange is super important becouse water from north sea is much saltier, colder and better oxygenated. Those exchanged are happening less and less often. Around 5-7 infusions per 10 years in 70-80's. While only two were observed sice 2000's. Furthermore every day sweet waters from more than 200 rivers are injected into the baltic sea. Which combined can turn baltic sea into freshwater tank.

Continuing on water exchange. The less often they occur, the higher  temperature should be. Temperature is one of things that promotes the growth of microorganisms and algae


In [ ]:
##TODO plot temperature to check if true best if using mean by season
#plot_depth(salinity,'CTDSAL', [4,5,6], 100)
df1 = filter_by_depth(ctdtmp, 100)
df1 = drop_outliers(df1, 'CTDTMP', .95)
df1 = group_by_year(df1, 'CTDTMP')

df2 = filter_by_depth(ctdtmp, 500)
df2 = drop_outliers(df2, 'CTDTMP', .95)
df2 = group_by_year(df2, 'CTDTMP')

df3 = filter_by_depth(ctdtmp, 1000)
df3 = drop_outliers(df3, 'CTDTMP', .95)
df3 = group_by_year(df3, 'CTDTMP')

df4 = filter_by_depth(ctdtmp, 2000)
df4 = drop_outliers(df4, 'CTDTMP', .95)
df4 = group_by_year(df4, 'CTDTMP')

#df = filter_by_months(df, [12])

#plt.ylim(5, 14)
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, figsize=(10,6))
ax1, ax2, ax3, ax4 = axes.flatten()
sns.scatterplot(data=df1, x="DATE", y='CTDTMP', ax = ax1)
sns.scatterplot(data=df2, x="DATE", y='CTDTMP', ax = ax2)
sns.scatterplot(data=df3, x="DATE", y='CTDTMP', ax = ax3)
sns.scatterplot(data=df4, x="DATE", y='CTDTMP', ax = ax4)
#plt.ylim(5,15)
plt.show()


## Industrial and agricultural activities

Industrial and agricultural activities aruond baltic sea 
causes the associated nutrient loads to end up in marine waters. Intensive agricultural production requires the use of fertilizers, in addition, farmers produce tons of organic waste, meat plants where thousands of pigs are raised using the mulchless method produce hectoliters of manure with a high content of minerals, especially..
#### nitrates
now farmers use 10 times less fertilizers than centuries ago, which actually can be seen in data, (visibly less nitrates)

In [ ]:
# TODO plot and analyse nitrates
df1 = filter_by_depth(TN, 100)
df1 = drop_outliers(df1, 'TN', .98)
df1 = group_by_year(df1, 'TN')

df2 = filter_by_depth(TN, 500)
df2 = drop_outliers(df2, 'TN', .98)
df2 = group_by_year(df2, 'TN')

df3 = filter_by_depth(TN, 1000)
df3 = drop_outliers(df3, 'TN', .98)
df3 = group_by_year(df3, 'TN')

df4 = filter_by_depth(TN, 2000)
df4 = drop_outliers(df4, 'TN', .98)
df4 = group_by_year(df4, 'TN')

#df = filter_by_months(df, [12])

#plt.ylim(5, 14)
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, figsize=(10,6))
ax1, ax2, ax3, ax4 = axes.flatten()
sns.scatterplot(data=df1, x="DATE", y='TN', ax = ax1)
sns.scatterplot(data=df2, x="DATE", y='TN', ax = ax2)
sns.scatterplot(data=df3, x="DATE", y='TN', ax = ax3)
sns.scatterplot(data=df4, x="DATE", y='TN', ax = ax4)
#plt.ylim(5,15)
plt.show()

In [ ]:
# TODO plot and analyse phosphates
df1 = filter_by_depth(TP, 100)
df1 = drop_outliers(df1, 'TP', .98)
df1 = group_by_year(df1, 'TP')

df2 = filter_by_depth(TP, 500)
df2 = drop_outliers(df2, 'TP', .98)
df2 = group_by_year(df2, 'TP')

df3 = filter_by_depth(TP, 1000)
df3 = drop_outliers(df3, 'TP', .98)
df3 = group_by_year(df3, 'TP')

df4 = filter_by_depth(TP, 2000)
df4 = drop_outliers(df4, 'TP', .98)
df4 = group_by_year(df4, 'TP')

#df = filter_by_months(df, [12])

#plt.ylim(5, 14)
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, figsize=(10,6))
ax1, ax2, ax3, ax4 = axes.flatten()
sns.scatterplot(data=df1, x="DATE", y='TP', ax = ax1)
sns.scatterplot(data=df2, x="DATE", y='TP', ax = ax2)
sns.scatterplot(data=df3, x="DATE", y='TP', ax = ax3)
sns.scatterplot(data=df4, x="DATE", y='TP', ax = ax4)
#plt.ylim(5,15)
plt.show()

to much of fertilizers is leading to overfertilization and algae blooms.
After war farmers began to use heavy laods of fertilizers causing in better growth of plants. No one was predicting that huge amounts of fertilizers will seep into the groundwater from there to the rivers and then to the sea.

And because as we said previously, baltic is badly communcated with global ocean it cleans up slowly

Causes of that are heavy growth of algae and cyanobacteria. 

The worst situation is from July to September, when cyanobateria called cyanobacteria bloom, poisoning the fish and cutting off the light to plants growing on the bottom.

In [ ]:
## TODO check disk visbility plot chlora levels, plot disk, visiblity 

df1 = filter_by_depth(secchi, 100)
df1 = drop_outliers(df1, 'SECCHI', .99)
df1 = filter_by_months(df1, [7,8,9])
#df1 = group_by_year(df1, 'SECCHI')

df2 = filter_by_depth(secchi, 500)
df2 = drop_outliers(df2, 'SECCHI', .99)
df2 = filter_by_months(df2, [7,8,9])
#df2 = group_by_year(df2, 'SECCHI')

df3 = filter_by_depth(secchi, 1000)
df3 = drop_outliers(df3, 'SECCHI', .99)
df3 = filter_by_months(df3, [7,8,9])
#df3 = group_by_year(df3, 'SECCHI')

df4 = filter_by_depth(secchi, 2000)
df4 = drop_outliers(df4, 'SECCHI', .99)
df4 = filter_by_months(df4, [7,8,9])
#df4 = group_by_year(df4, 'SECCHI')

#df = filter_by_months(df, [12])

#plt.ylim(5, 14)
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, figsize=(10,6))
ax1, ax2, ax3, ax4 = axes.flatten()
sns.scatterplot(data=df1, x="DATE", y='SECCHI', ax = ax1)
sns.scatterplot(data=df2, x="DATE", y='SECCHI', ax = ax2)
sns.scatterplot(data=df3, x="DATE", y='SECCHI', ax = ax3)
sns.scatterplot(data=df4, x="DATE", y='SECCHI', ax = ax4)
#plt.ylim(5,15)
plt.show()

#ok there seems to be less light based on disk visibility

## OXYGEN

### Continuing on problems we explained 

(in general)  Dissolved oxygen comes from air and partialy from photosynthesis 
(this is a general theory because the one in the baltic, as has already been said, should also be injected from the northern sea)

Increased eutrophication and climate change as a result of human economic activities contribute to the formation of dead zones. Where 
### Oxygen
is lacking encapsulated phosphorus is released from bottom sediments and further increases the concentration of nutrients

In [ ]:
## plot oxygen, plot phoshorus levels and micronutrients
# TODO plot and analyse nitrates
df1 = filter_by_depth(oxygen, 100)
df1 = drop_outliers(df1, 'OXY', .99)
df1 = filter_by_months(df1, [7,8,9])
#df1 = group_by_year(df1, 'OXY')

df2 = filter_by_depth(oxygen, 1000)
df2 = drop_outliers(df2, 'OXY', .99)
df2 = filter_by_months(df2, [7,8,9])
#df2 = group_by_year(df2, 'OXY')

df3 = filter_by_depth(oxygen, 1500)
df3 = drop_outliers(df3, 'OXY', .99)
df3 = filter_by_months(df3, [7,8,9])
#df3 = group_by_year(df3, 'OXY')

df4 = filter_by_depth(oxygen, 2000)
df4 = drop_outliers(df4, 'OXY', .99)
df4 = filter_by_months(df4, [7,8,9])
#df4 = group_by_year(df4, 'OXY')


#plt.ylim(5, 14)
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, figsize=(10,6))
ax1, ax2, ax3, ax4 = axes.flatten()
sns.scatterplot(data=df1, x="DATE", y='OXY', ax = ax1)
sns.scatterplot(data=df2, x="DATE", y='OXY', ax = ax2)
sns.scatterplot(data=df3, x="DATE", y='OXY', ax = ax3)
sns.scatterplot(data=df4, x="DATE", y='OXY', ax = ax4)
#plt.ylim(5,15)
plt.show()

As their amount grow and the water heats up, the algae develops faster and the water blooms more intensively.

### Further info on chlora and oxygen

Oxygen is used to decompose organic matter that settles to the bottom. Lacking oxygen which should be delivered by north sea there is no regeneration process. 

Oxygen is also essential for marine organisms. At least 6 mg/l dissolved oxygen is needed for life. On the surface of the open Baltic Sea it is about 11 mg/l, but it is worse at the bottom, where hypoxia is recorded as low as 2 mg/l.

Degradation began at the beginning of the 20th century, even a hundred years ago there were zones with reduced oxygen content, but in the last century they appeared 12 times more.

furthermore 
### phytoplankton

whose development is fostered by the above-mentioned agricultural processes leads to the formation of large amounts of plant debris falling to the bottom there itself feeding microorganisms that take oxygen away from fish and other animals. When the resources are used up, anaerobic bacteria appear, causing rotting processes and emitting poisonous hydrogen sulfide, which turns the water at the bottom into stinking mud.


Side note 

Another problem you hear about are the tons of sunken munitions and chemical weapons left over from the 2nd Swastika War (about 50,000 tons) including sulfur mustard. Fortunately, they are in the form of a very poorly water-soluble thick slurry, so left alone they do not pose much of a threat.

##### some reference
https://www.teraz-srodowisko.pl/aktualnosci/Baltyk-eutrofizacja-morski-ekosystem-rolnictwo-scieki-11653.html
https://www.newsweek.pl/zdrowie-i-nauka/nauka/morze-baltyckie-zmienia-sie-w-zielone-pelne-szlamu-bajoro/6fews3b
https://ekoagora.pl/wp-content/uploads/2021/05/PL-APC-40-3.pdf

In [ ]:
zip_file = ZipFile('./2022.11.07-merged-single-observation.zip')
dfs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()
       if text_file.filename.endswith('.csv') and not text_file.filename[37:].startswith('.')}

df_list=[]
for df in dfs.values():
       df['DATE'] = pd.to_datetime(df['DATE'],dayfirst = True)
       df_list.append(df)

df = reduce(lambda left,right: pd.merge(left.drop_duplicates(subset=['DATE','DEPTH']),right.drop_duplicates(subset=['DATE','DEPTH']),on=['DATE','DEPTH'],how='outer'), df_list)
# df.to_csv('xxxxx.csv')
df.head

## Correlation

In [ ]:
dfi = df
dfi = dfi.drop('DEPTH', axis=1)
fig = plt.figure(figsize=(15,15))
sns.heatmap(dfi.corr(),vmax=1, annot=True, linewidths=0.5, cbar=False, cmap='YlGnBu',annot_kws={'fontsize':14})
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.title('xx',fontsize=14)
plt.show()

In [ ]:
dfi = df
dfi.dropna(inplace=True)
dfi = dfi.drop('DEPTH', axis=1)
fig = plt.figure(figsize=(15,15))
sns.heatmap(dfi.corr(),vmax=1, annot=True, linewidths=0.5, cbar=False, cmap='YlGnBu',annot_kws={'fontsize':18})
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.title('xx',fontsize=14)
plt.show()

## By depth

In [ ]:

for i in depths:
    dfi = filter_by_depth(df, i)
    dfi = dfi.drop('DEPTH', axis=1)
    fig = plt.figure(figsize=(15,15))
    sns.heatmap(dfi.corr(),vmax=1, annot=True, linewidths=0.5, cbar=False, cmap='YlGnBu',annot_kws={'fontsize':14})
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.title('xx',fontsize=14)
    plt.title(f"depth : {i}")
    plt.show()

## Exploring correlations found in data further

In [ ]:
def plot_params(df, parameter1, parameter2, depths, quant=.95):
    for i in depths:
            df_ = filter_by_depth(df,i)
            df_ = drop_outliers(df_, parameter1, quant)
            df_ = drop_outliers(df_, parameter2, quant)
            
            plt.figure(figsize=(12,8))
            plt.subplot(2,2,1)
            sns.scatterplot(data=df_, x=parameter1, y=parameter2)
            plt.title(f'{parameter1} & {parameter2} \n DEPTH: {i}')


In [ ]:

plot_params(df, 'NO2', 'NH4', [100,500,1000])
plot_params(df, 'NO3', 'NH4', [100,500])
plot_params(df, 'NO3', 'NO2', [100,500,1000,1500])
plot_params(df, 'OXY', 'CHLORA', [100])
plot_params(df, 'OXY', 'CTDTMP', [100,500,1000])

plot_params(df, 'OXY', 'SECCHI', [100])
plot_params(df, 'OXY', 'CTDTMP', [2000])
plot_params(df, 'OXY', 'SIO2', [2000,2500])
plot_params(df, 'OXY', 'PO4', [2000,2500])
plot_params(df, 'OXY', 'TP', [2500])
plot_params(df, 'PO4', 'NH4', [100,500,1000])
plot_params(df, 'PO4', 'NO2', [100,1000])
plot_params(df, 'PO4', 'NO3', [100,500,1000])
plot_params(df, 'PO4', 'SIO2', [2000])
plot_params(df, 'TP', 'SIO2', [100,500,1000,1500,2000,2500])
plot_params(df, 'TP', 'PO4', [100,500,1000,1500,2000,2500])
plot_params(df, 'TP', 'CTDSAL', [1000])
plot_params(df, 'TP', 'NH4', [1000])
plot_params(df, 'TP', 'NO2', [1000])
plot_params(df, 'TP', 'CTDTMP', [2500])
plot_params(df, 'SECCHI', 'CTDTMP', [2000])
plot_params(df, 'TN', 'NH4', [2500])


### SUMMARY:
 NO3 & NO2 visible linear correlation, no surprise
 
 OXY & CTDTMP visible negative linear correlation  it's just water property
 
 OXY & SIO2 could be interesting
 
 OXY & PO4 also interesting
 
 TP & SIO2 also interesting
 
 TP & PO4 visible linear correlation, no surprise

## Measure average by depth discarding date

In [ ]:
chlora_by_depth = group_by_depth(chlora, 'CHLORA')
sns.scatterplot(data=chlora_by_depth, x="DEPTH", y='CHLORA')

In [ ]:
ctdsal_by_depth = group_by_depth(ctdsal, 'CTDSAL')
sns.scatterplot(data=ctdsal_by_depth, x="DEPTH", y='CTDSAL')

In [ ]:
ctdtmp_by_depth = group_by_depth(ctdtmp, 'CTDTMP')
sns.scatterplot(data=ctdtmp_by_depth, x="DEPTH", y='CTDTMP')

In [ ]:
ammonia_by_depth = group_by_depth(NH4, 'NH4')
sns.scatterplot(data=ammonia_by_depth, x="DEPTH", y='NH4')

In [ ]:
nitrogen_by_depth = group_by_depth(NO2, 'NO2')
sns.scatterplot(data = nitrogen_by_depth, x="DEPTH", y='NO2')

In [ ]:
nitrate_by_depth = group_by_depth(NO3, 'NO3')
sns.scatterplot(data=nitrate_by_depth, x="DEPTH", y='NO3')

In [ ]:
oxygen_by_depth = group_by_depth(oxygen, 'OXY')
sns.scatterplot(data=oxygen_by_depth, x="DEPTH", y='OXY')

In [ ]:
ph_by_depth = group_by_depth(PH, 'PH')
sns.scatterplot(data=ph_by_depth, x="DEPTH", y='PH')

In [ ]:
phosphate_by_depth = group_by_depth(PO4, 'PO4')
sns.scatterplot(data=phosphate_by_depth, x="DEPTH", y='PO4')

In [ ]:
secchi_by_depth = group_by_depth(secchi, 'SECCHI')
sns.scatterplot(data=secchi_by_depth, x="DEPTH", y='SECCHI')

In [ ]:
silicon_by_depth = group_by_depth(SIO2, 'SIO2')
sns.scatterplot(data=silicon_by_depth, x="DEPTH", y='SIO2')

In [ ]:
nitrogen_by_depth = group_by_depth(TN, 'TN')
sns.scatterplot(data=nitrogen_by_depth, x="DEPTH", y='TN')

In [ ]:
phosphorus_by_depth = group_by_depth(TP, 'TP')
sns.scatterplot(data=phosphorus_by_depth, x="DEPTH", y='TP')

In [ ]:
# plot parameter from df by different months and deepths to search manually
def check_manualy(df, parameter, quant=.99):
    for i in depths:
        for j in range(1,13):
            df_ = drop_outliers(df, parameter, quant)
            df_ = filter_by_depth(df_,i)
            df_ = filter_by_months(df_, [j])
            #df.groupby('DATE').mean()['TP'].plot()
            plt.title(f"month : {j}  depth : {i}")
            sns.scatterplot(data=df_, x="DATE", y=parameter)
            plt.pause(0.05)

def check_yearly(df, parameter, quant=.99):
    for i in depths:
        df_ = filter_by_depth(df,i)
        df_ = group_by_year(df_, parameter)
        plt.title(f"depth : {i}")
        sns.scatterplot(data=df_, x="DATE", y=parameter)
        plt.pause(0.05)

# now let's see which parameters changed by date

## CHLORA, seems to be decreasing looking at yearly avg

In [ ]:
check_manualy(chlora, 'CHLORA')

In [ ]:

check_yearly(chlora, 'CHLORA')

## Salinity seems to be increasing at 1km and above looking at monthly's

In [ ]:
check_manualy(ctdsal, 'CTDSAL')

In [ ]:
check_yearly(ctdsal, 'CTDSAL')

## Water temperature at measurement depth, increasing

In [ ]:
check_manualy(ctdtmp, 'CTDTMP')

In [ ]:
check_yearly(ctdtmp, 'CTDTMP')

## Ammonia decreasing since around 1995, increasing by a bit before

In [ ]:
check_manualy(NH4, 'NH4')

In [ ]:
check_yearly(NH4, 'NH4')

## Nitrogen dioxide, decreasing

In [ ]:
check_manualy(NO2, 'NO2')

In [ ]:
check_yearly(NO2, 'NO2')

## Nitrate decreasing

In [ ]:
check_manualy(NO3, 'NO3')

In [ ]:
check_yearly(NO3, 'NO3')

## Dissolved oxygen, decreasing till 2010 at 2,5 km above still decreasing

In [ ]:
check_manualy(oxygen, 'OXY')

In [ ]:
check_yearly(oxygen, 'OXY')

## PH, hard to tell, no changes prob.

In [ ]:
check_manualy(PH, 'PH')

In [ ]:
check_yearly(PH, 'PH')

## Phosphate increasing till around 1990 then started decreasing, at 2km and above, decreasing below


In [ ]:
check_manualy(PO4,'PO4')

check_yearly(PO4, 'PO4')

## SECCHI decreasing

In [ ]:
check_manualy(secchi, 'SECCHI')

In [ ]:
check_yearly(secchi, 'SECCHI')

## SILICATE, visible pattern at 2,5 km  increasing till around 2008 then decreasing, hard to tell at other depths

In [ ]:
check_manualy(SIO2, 'SIO2')

In [ ]:
check_yearly(SIO2, 'SIO2')

## Total nitrogen, clear pattern of droping since around 2010 at each depth

In [ ]:
check_manualy(TN, 'TN')

In [ ]:
check_yearly(TN, 'TN')

## Total phosphorus looks rather constant

In [ ]:
check_manualy(TP, 'TP')

In [ ]:
check_yearly(TP, 'TP')